In [1]:
!pip install calplot pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for calplot: filename=calplot-0.1.7.5-py3-none-any.whl size=8115 sha256=1cc3a52bef9b4ef2beff495d5dbca47dbe8440ad23268b5ad0e424937f069b69
  Stored in directory: /root/.cache/pip/wheels/77/75/32/a518a4a1421776a93e1ede33cb8dac474095bf31fed7e4e22c
Successfully built calplot


In [2]:
import pandas as pd

#Plot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import calendar
import calplot # actually used

# Score model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

In [3]:
url = "https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/main/ProjectAssignmentData/Dataset-PT.csv"
df = pd.read_csv(url,header=1)

In [4]:
df.head(5)
# df.size 545104 rows

,Calendar_date,route_id,bus_id,stop_sequence,arrival_delay,dwell_time,travel_time_for_previous_section,scheduled_travel_time,upstream_stop_delay,origin_delay,...,factor(weather)Rain,factor(weather)Snow,factor(temperature)Cold,factor(temperature)Extra_cold,factor(temperature)Normal,factor(day_of_week)weekday,factor(day_of_week)weekend,factor(time_of_day)Afternoon_peak,factor(time_of_day)Morning_peak,factor(time_of_day)Off-peak
0,20220108,4,41344,1,151,0,0,120,100,100,...,0,0,0,0,1,0,1,0,0,1
1,20220108,4,41344,2,185,24,171,45,151,100,...,0,0,0,0,1,0,1,0,0,1
2,20220108,4,41344,3,186,0,55,41,185,100,...,0,0,0,0,1,0,1,0,0,1
3,20220108,4,41344,4,202,12,42,94,186,100,...,0,0,0,0,1,0,1,0,0,1
4,20220108,4,41344,5,242,21,98,86,202,100,...,0,0,0,0,1,0,1,0,0,1


In [5]:
from scipy import stats

# Calculate z-scores for the 'arrival_delay' column
z_scores = stats.zscore(df['arrival_delay'])

# Get boolean array indicating the location of outliers
outliers = (z_scores > 7) | (z_scores < -7)

# Count the number of outliers
num_outliers = outliers.sum()

# Print the number of outliers
print(f"Number of outliers removed: {num_outliers}")

# Remove the outliers
df_no_outliers = df[~outliers]

# Verify the new size of the DataFrame
print(f"Size of the original DataFrame: {len(df)}")
print(f"Size of the DataFrame after removing outliers: {len(df_no_outliers)}")

Number of outliers removed: 228
Size of the original DataFrame: 545103
Size of the DataFrame after removing outliers: 544875


In [40]:
df = df_no_outliers

# Model without weather

In [41]:
desired_stop_sequences = [1, 5, 8, 12, 14, 16, 20, 21, 23]

# desired_stop_sequences = [1, 5]  # In case to verify the codes

# Create a boolean mask for rows with the desired stop sequence
mask = df_no_outliers['stop_sequence'].isin(desired_stop_sequences)

# Filter the DataFrame to keep only the rows with the desired stop sequence
df_selected_stops = df_no_outliers[mask]

df_selected_stops.shape

(181626, 31)

In [42]:
df_9stops = df_selected_stops.drop(['Calendar_date','route_id', 'bus_id'], axis = 1)
columns_to_drop = [
    'weather', 'temperature', 'time_of_day',
    'factor(weather)Light_Rain', 'factor(weather)Light_Snow',
    'factor(weather)Normal', 'factor(weather)Rain', 'factor(weather)Snow',
    'factor(temperature)Cold', 'factor(temperature)Extra_cold',
    'factor(temperature)Normal', 'factor(day_of_week)weekday',
    'factor(day_of_week)weekend', 'factor(time_of_day)Afternoon_peak',
    'factor(time_of_day)Morning_peak', 'factor(time_of_day)Off-peak'
]
for column in columns_to_drop:
    df_9stops.pop(column)

In [43]:
df_9stops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181626 entries, 0 to 545098
Data columns (total 12 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   stop_sequence                     181626 non-null  int64  
 1   arrival_delay                     181626 non-null  int64  
 2   dwell_time                        181626 non-null  int64  
 3   travel_time_for_previous_section  181626 non-null  int64  
 4   scheduled_travel_time             181626 non-null  int64  
 5   upstream_stop_delay               181626 non-null  int64  
 6   origin_delay                      181626 non-null  int64  
 7   previous_bus_delay                181626 non-null  int64  
 8   previous_trip_travel_time         181626 non-null  int64  
 9   traffic_condition                 181626 non-null  float64
 10  recurrent_delay                   181626 non-null  float64
 11  day_of_week                       181626 non-null  o

### Neural Network Model

In [44]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_absolute_error
import joblib
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

def train_and_evaluate_nn(df):
    # Split the dataset into training and test sets
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

    if df_train.empty or df_test.empty:
        print("Training or testing dataset is empty.")
        return None  # Return early if the dataset is empty

    sc = StandardScaler()

    y_mean = df_test['arrival_delay'].mean()
    overall_ssr = 0  # Initialize overall_ssr
    overall_tss = 0  # Initialize overall_tss
    overall_mae_sum = 0  # Initialize overall MAE sum
    total_predictions = 0  # Initialize total predictions

    results = {
        'stop_sequence': [],
        'day_type': [],
        'R^2': [],
        'MAE': []
    }

    for stop_seq in df['stop_sequence'].unique():
        for day_type in ['weekday', 'weekend']:
            # Filter data
            df_train_subset = df_train[(df_train['stop_sequence'] == stop_seq) & (df_train['day_of_week'] == day_type)]
            df_test_subset = df_test[(df_test['stop_sequence'] == stop_seq) & (df_test['day_of_week'] == day_type)]

            # Prepare data
            x_train = df_train_subset.drop(['arrival_delay', 'day_of_week'], axis=1)
            y_train = df_train_subset['arrival_delay']

            x_test = df_test_subset.drop(['arrival_delay', 'day_of_week'], axis=1)
            y_test = df_test_subset['arrival_delay']

            # Normalize the input features
            x_train = sc.fit_transform(x_train)
            x_test = sc.transform(x_test)

            # Neural network model
            model = Sequential()
            model.add(Dense(32, activation='linear', input_dim=x_train.shape[1]))
            model.add(Dropout(0.001))
            model.add(Dense(64, activation='linear'))
            model.add(Dense(1))

            model.compile(optimizer='adam', loss='mae', metrics=['mae'])

            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            model.fit(x_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping], verbose=1)

            y_pred = model.predict(x_test).flatten()
            current_r2 = r2_score(y_test, y_pred)
            current_mae = mean_absolute_error(y_test, y_pred)

            residuals = y_test - y_pred
            ssr = sum(residuals**2)
            overall_ssr += ssr
            tss = sum((y_test - y_mean)**2)
            overall_tss += tss

            total_predictions += len(y_test)
            overall_mae_sum += current_mae * len(y_test)

            results['stop_sequence'].append(stop_seq)
            results['day_type'].append(day_type)
            results['R^2'].append(current_r2)
            results['MAE'].append(current_mae)

    overall_r2 = 1 - (overall_ssr / overall_tss)
    overall_mae = overall_mae_sum / total_predictions  # Calculate overall MAE

    print(f'Overall R^2 for Neural Network: {overall_r2}')
    print(f'Overall MAE for Neural Network: {overall_mae}')

    return pd.DataFrame(results)

# Example usage:
results_df = train_and_evaluate_nn(df_9stops)
if results_df is not None:
    print(results_df)


Epoch 1/100
309/309 [==============================] - 2s 3ms/step - loss: 47.8543 - mae: 47.8543 - val_loss: 27.6448 - val_mae: 27.6448
Epoch 2/100
309/309 [==============================] - 1s 4ms/step - loss: 27.3806 - mae: 27.3806 - val_loss: 27.5837 - val_mae: 27.5837
Epoch 3/100
309/309 [==============================] - 1s 5ms/step - loss: 27.3942 - mae: 27.3942 - val_loss: 27.6158 - val_mae: 27.6158
Epoch 4/100
309/309 [==============================] - 1s 3ms/step - loss: 27.3734 - mae: 27.3734 - val_loss: 27.6253 - val_mae: 27.6253
Epoch 5/100
309/309 [==============================] - 1s 3ms/step - loss: 27.4083 - mae: 27.4083 - val_loss: 27.8424 - val_mae: 27.8424
Epoch 6/100
309/309 [==============================] - 1s 2ms/step - loss: 27.3820 - mae: 27.3820 - val_loss: 27.6290 - val_mae: 27.6290
Epoch 7/100
309/309 [==============================] - 1s 3ms/step - loss: 27.3341 - mae: 27.3341 - val_loss: 27.5494 - val_mae: 27.5494
Epoch 8/100
309/309 [====================

### Regression Model

In [45]:
df_9stops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181626 entries, 0 to 545098
Data columns (total 12 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   stop_sequence                     181626 non-null  int64  
 1   arrival_delay                     181626 non-null  int64  
 2   dwell_time                        181626 non-null  int64  
 3   travel_time_for_previous_section  181626 non-null  int64  
 4   scheduled_travel_time             181626 non-null  int64  
 5   upstream_stop_delay               181626 non-null  int64  
 6   origin_delay                      181626 non-null  int64  
 7   previous_bus_delay                181626 non-null  int64  
 8   previous_trip_travel_time         181626 non-null  int64  
 9   traffic_condition                 181626 non-null  float64
 10  recurrent_delay                   181626 non-null  float64
 11  day_of_week                       181626 non-null  o

In [46]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import RFECV

def train_and_evaluate(df):
    # Split the entire dataset into training and test sets
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

    y_mean = df_test['arrival_delay'].mean()

    results = {
        'stop_sequence': [],
        'day_type': [],
        'R^2': [],
        'MAE': []
    }

    weighted_mae_sum = 0
    total_samples = 0
    overall_ssr = 0
    overall_tss = 0

    for stop_seq in df['stop_sequence'].unique():
        for day_type in ['weekday', 'weekend']:
            # Filter data by stop sequence and day type
            df_train_subset = df_train[(df_train['stop_sequence'] == stop_seq) & (df_train['day_of_week'] == day_type)]
            df_test_subset = df_test[(df_test['stop_sequence'] == stop_seq) & (df_test['day_of_week'] == day_type)]

            # Train model with RFECV
            x_train = df_train_subset.drop(['arrival_delay', 'day_of_week'], axis=1)
            y_train = df_train_subset['arrival_delay']

            model = LinearRegression()
            selector = RFECV(estimator=model, step=1, cv=KFold(5))
            selector = selector.fit(x_train, y_train)

            # Fit model with selected features
            model.fit(x_train.iloc[:, selector.support_], y_train)

            # Evaluate the model
            x_test = df_test_subset.drop(['arrival_delay', 'day_of_week'], axis=1).iloc[:, selector.support_]
            y_test = df_test_subset['arrival_delay']

            y_pred = model.predict(x_test)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            sample_count = len(y_test)
            weighted_mae_sum += mae * sample_count
            total_samples += sample_count

            residuals = y_test - y_pred
            ssr = sum(residuals**2)
            overall_ssr += ssr
            tss = sum((y_test - y_mean)**2)
            overall_tss += tss

            results['stop_sequence'].append(stop_seq)
            results['day_type'].append(day_type)
            results['R^2'].append(r2)
            results['MAE'].append(mae)


    overall_r2 = 1 - (overall_ssr / overall_tss)
    overall_mae = weighted_mae_sum / total_samples

    print(f'Overall R^2: {overall_r2}')
    print(f'Overall MAE: {overall_mae}')

    return pd.DataFrame(results)

# Example usage:
results_df = train_and_evaluate(df_9stops)
print(results_df)

Overall R^2: 0.986543559614433
Overall MAE: 14.639100285929766
    stop_sequence day_type       R^2        MAE
0               1  weekday  0.895897  29.206868
1               1  weekend  0.950604  27.460467
2               5  weekday  0.977695  14.720359
3               5  weekend  0.991448  11.532659
4               8  weekday  0.981402  15.946810
5               8  weekend  0.991961  12.834797
6              12  weekday  0.990878  12.361416
7              12  weekend  0.995921  10.499564
8              14  weekday  0.987187  14.936114
9              14  weekend  0.992177  15.715836
10             16  weekday  0.977038  20.886223
11             16  weekend  0.992190  15.197998
12             20  weekday  0.994707   9.459281
13             20  weekend  0.997881   7.699327
14             21  weekday  0.992222  13.044044
15             21  weekend  0.995861  11.226847
16             23  weekday  0.998542   5.370143
17             23  weekend  0.999086   4.747134


### LR VS NN

In [47]:
df_9stops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181626 entries, 0 to 545098
Data columns (total 12 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   stop_sequence                     181626 non-null  int64  
 1   arrival_delay                     181626 non-null  int64  
 2   dwell_time                        181626 non-null  int64  
 3   travel_time_for_previous_section  181626 non-null  int64  
 4   scheduled_travel_time             181626 non-null  int64  
 5   upstream_stop_delay               181626 non-null  int64  
 6   origin_delay                      181626 non-null  int64  
 7   previous_bus_delay                181626 non-null  int64  
 8   previous_trip_travel_time         181626 non-null  int64  
 9   traffic_condition                 181626 non-null  float64
 10  recurrent_delay                   181626 non-null  float64
 11  day_of_week                       181626 non-null  o

In [53]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import os

def train_evaluate_best_model(df, n_folds=5):

    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

    results = {
        'stop_sequence': [],
        'day_type': [],
        'best_model': [],
        'R^2': [],
        'MAE': []
    }

    overall_r2_values = []
    overall_sample_weights = []

    for stop_seq in df['stop_sequence'].unique():
        for day_type in ['weekday', 'weekend']:
            df_train_subset = df_train[(df_train['stop_sequence'] == stop_seq) & (df_train['day_of_week'] == day_type)]
            df_test_subset = df_test[(df_test['stop_sequence'] == stop_seq) & (df_test['day_of_week'] == day_type)]

            drop_columns = ['arrival_delay', 'day_of_week']
            x_train = df_train_subset.drop(columns=drop_columns, errors='ignore').astype('float32')
            y_train = df_train_subset['arrival_delay'].astype('float32')
            x_test = df_test_subset.drop(columns=drop_columns, errors='ignore').astype('float32')
            y_test = df_test_subset['arrival_delay'].astype('float32')

            r2_scores_nn = []
            r2_scores_lr = []
            maes_nn = []
            maes_lr = []

            # K-fold CV
            kf = KFold(n_splits=n_folds)
            for train_index, val_index in kf.split(x_train):
                x_train_fold, x_val_fold = x_train.iloc[train_index], x_train.iloc[val_index]
                y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

                # Train and evaluate NN
                model_nn = Sequential([
                    Dense(32, activation='linear', input_dim=x_train_fold.shape[1]),
                    Dropout(0.001),
                    Dense(64, activation='linear'),
                    Dense(1)
                ])
                model_nn.compile(optimizer='adam', loss='mae', metrics=['mae'])
                early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
                model_nn.fit(x_train_fold, y_train_fold, validation_data=(x_val_fold, y_val_fold), epochs=100, batch_size=32, callbacks=[early_stopping], verbose=0)
                y_pred_nn = model_nn.predict(x_val_fold).flatten()
                r2_scores_nn.append(r2_score(y_val_fold, y_pred_nn))
                maes_nn.append(mean_absolute_error(y_val_fold, y_pred_nn))

                # Train and evaluate LR
                model_lr = LinearRegression().fit(x_train_fold, y_train_fold)
                y_pred_lr = model_lr.predict(x_val_fold)
                r2_scores_lr.append(r2_score(y_val_fold, y_pred_lr))
                maes_lr.append(mean_absolute_error(y_val_fold, y_pred_lr))

            # Average scores
            avg_r2_nn = np.mean(r2_scores_nn)
            avg_r2_lr = np.mean(r2_scores_lr)
            avg_mae_nn = np.mean(maes_nn)
            avg_mae_lr = np.mean(maes_lr)

            # Determine best model
            if avg_r2_nn > avg_r2_lr:
                best_model = "NN"
                best_r2 = avg_r2_nn
                best_mae = avg_mae_nn
            else:
                best_model = "LR"
                best_r2 = avg_r2_lr
                best_mae = avg_mae_lr

            # Save models
            model_dir = "models"
            if not os.path.exists(model_dir):
                os.makedirs(model_dir)
            if best_model == "NN":
                model_nn.save(os.path.join(model_dir, f'nn_model_{stop_seq}_{day_type}.h5'))
            else:
                joblib.dump(model_lr, os.path.join(model_dir, f'lr_model_{stop_seq}_{day_type}.pkl'))

            overall_r2_values.append(best_r2)
            overall_sample_weights.append(len(df_test_subset))
            results['stop_sequence'].append(stop_seq)
            results['day_type'].append(day_type)
            results['best_model'].append(best_model)
            results['R^2'].append(best_r2)
            results['MAE'].append(best_mae)

    overall_r2 = np.average(overall_r2_values, weights=overall_sample_weights)
    print(f"Overall R^2: {overall_r2:.2f}")

    overall_mae = np.average(results['MAE'], weights = overall_sample_weights)
    print(f"Overall MAE: {overall_mae:.2f}")

    return pd.DataFrame(results)

# predict_new_data function remains mostly the same.
def predict_new_data(df_9stops):
    all_groups = []  # Placeholder for updated groups

    # Group the dataframe by 'stop_sequence' and 'day_of_week'
    grouped = df.groupby(['stop_sequence', 'day_of_week'])

    for (stop_seq, day_type), group in grouped:
        nn_model_path = os.path.join("models", f'nn_model_{stop_seq}_{day_type}.h5')
        lr_model_path = os.path.join("models", f'lr_model_{stop_seq}_{day_type}.pkl')

        X = group.drop(columns=['arrival_delay', 'day_of_week'], errors='ignore').astype('float32')

        if os.path.exists(nn_model_path):
            model = load_model(nn_model_path)
            group['predicted_delay'] = model.predict(X).flatten()

        elif os.path.exists(lr_model_path):
            model = joblib.load(lr_model_path)
            group['predicted_delay'] = model.predict(X)

        else:
            raise ValueError(f"No saved model found for stop_sequence: {stop_seq} and day_type: {day_type}")

        all_groups.append(group)

    # Concatenate all updated groups to get the final dataframe
    result_df = pd.concat(all_groups)

    return result_df

In [54]:
results_df = train_evaluate_best_model(df_9stops)
print(results_df)

24/24 [==============================] - 0s 2ms/step
Overall R^2: 0.98
Overall MAE: 14.53
    stop_sequence day_type best_model       R^2        MAE
0               1  weekday         LR  0.903063  28.628769
1               1  weekend         LR  0.957562  27.048279
2               5  weekday         LR  0.978734  14.499735
3               5  weekend         LR  0.992105  11.614477
4               8  weekday         LR  0.976778  16.405163
5               8  weekend         LR  0.992977  12.610491
6              12  weekday         LR  0.990813  12.240393
7              12  weekend         LR  0.995987  10.468593
8              14  weekday         LR  0.987009  14.920743
9              14  weekend         LR  0.991046  16.113323
10             16  weekday         LR  0.975959  20.235369
11             16  weekend         LR  0.992163  14.783974
12             20  weekday         LR  0.995011   9.325909
13             20  weekend         LR  0.997587   8.078837
14             21  weekda